In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [3]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
      nn.Linear(28*28, 512),
      nn.ReLU(),
      nn.Linear(512, 512),
      nn.ReLU(),
      nn.Linear(512, 10),
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")
print(pred_probab)

Predicted class: tensor([8])
tensor([[0.1022, 0.0943, 0.0912, 0.1040, 0.0998, 0.1012, 0.0963, 0.0947, 0.1144,
         0.1019]], grad_fn=<SoftmaxBackward0>)


In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [15]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.2404, -0.0390, -0.3143, -0.5189, -0.0497,  0.2170,  0.0209, -0.0515,
         -0.4208, -0.1135,  0.3813,  0.4409, -0.0568,  0.6536,  0.2848,  0.1462,
          0.4102, -0.0430,  0.4304,  0.2483],
        [ 0.6096, -0.2623, -0.0996, -0.2614, -0.3111,  0.0932, -0.1403,  0.2350,
         -0.3298,  0.0814,  0.0049,  0.4620,  0.2676,  0.4632,  0.4582, -0.1066,
          0.1520, -0.1690,  0.2572,  0.2570],
        [ 0.4794, -0.3087, -0.3351, -0.5524,  0.0147,  0.6610, -0.2107, -0.0061,
         -0.4118,  0.0430,  0.3825,  0.4873,  0.2069,  0.6994,  0.1182,  0.0134,
          0.1333,  0.1982,  0.1218,  0.2610]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.2404, 0.0000, 0.0000, 0.0000, 0.0000, 0.2170, 0.0209, 0.0000, 0.0000,
         0.0000, 0.3813, 0.4409, 0.0000, 0.6536, 0.2848, 0.1462, 0.4102, 0.0000,
         0.4304, 0.2483],
        [0.6096, 0.0000, 0.0000, 0.0000, 0.0000, 0.0932, 0.0000, 0.2350, 0.0000,
         0.0814, 0.0049, 0.4620, 0.2676, 0.4632, 0.45

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [16]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 2.5247e-02,  3.2470e-02, -2.0555e-02,  ...,  2.2375e-02,
          2.7125e-02,  7.8354e-05],
        [ 1.5443e-02, -2.5244e-02, -2.8192e-02,  ..., -1.2313e-02,
          1.8113e-02, -1.9782e-03]], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0194, -0.0047], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 3.3829e-02,  2.2780e-02, -8.7375e-03,  ..., -3.2344e-02,
         -4.1507e-02,  6.7580e-03],
        [ 2.2493e-02, -7.3379e-03, -1.1739e-02, 